# Import libraries


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101, ResNet50, EfficientNetV2B0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.regularizers import l2
import pickle
from PIL import Image
import os
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
from keras_tuner import RandomSearch

# Setup and load data


In [ ]:
train_dir = '../equidata/split/train'
validation_dir = '../equidata/split/val'
test_dir = '../equidata/split/test'

BATCH_SIZE = 50
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(
    directory=train_dir,
    shuffle=True,
    seed=1337,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical'
)

validation_dataset = image_dataset_from_directory(
    directory=validation_dir,
    seed=1337,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical'
)

test_dataset = image_dataset_from_directory(
    directory=test_dir,
    seed=1337,
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    label_mode='categorical'
)

class_names = train_dataset.class_names

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Building and training model


In [ ]:
def build_model(hp):
    inputs = Input(shape=(224, 224, 3))

    lr = hp.Float('learning_rate', min_value=1e-5,
                  max_value=1e-2, sampling='log')
    base_model = EfficientNetV2B0(
        include_top=False, weights='imagenet', input_tensor=x)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(15, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', Precision(), Recall()])

    return model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=8,
    executions_per_trial=1,
    directory='random_search',
    project_name='album_cover_classification'
)

In [ ]:
tuner.search(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[checkpoint_callback]
)

# Fit model from best learning rate


In [ ]:
def build_model(hp):
    inputs = Input(shape=(224, 224, 3))

    lr = hp.Float('learning_rate', min_value=1e-5,
                  max_value=1e-2, sampling='log')
    base_model = EfficientNetV2B0(
        include_top=False, weights='imagenet', input_tensor=x)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(15, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', Precision(), Recall()])

    return model

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"{best_hps.get('learning_rate')}.")

optimal_lr = best_hps.get('learning_rate')
model = build_model(optimal_lr)

In [ ]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[checkpoint_callback]
)

with open('model_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# Load model and history


In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('best_model.h5')

In [ ]:
# Load history
import pickle
with open('model_history.pkl', 'rb') as f:
    history = pickle.load(f)

# Metrics


## Evaluation


In [ ]:
test_loss, test_acc, test_prec, test_recall = best_model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Precision: {test_prec*100:.2f}%")
print(f"Test Recall: {test_recall*100:.2f}%")

## Predictions


In [ ]:
predictions = best_model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.concatenate([y for x, y in test_dataset], axis=0)
true_classes = np.argmax(true_classes, axis=1)

## Compute F1 Score, Classification Report, and Confusion Matrix


In [ ]:
f1 = f1_score(true_classes, predicted_classes, average='weighted')
print(f"F1 Score: {f1:.2f}")

report = classification_report(
    true_classes, predicted_classes, target_names=class_names)
print(report)

conf_mat = confusion_matrix(true_classes, predicted_classes)

## Plot Confusion Matrix


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

conf_mat = confusion_matrix(true_classes, predicted_classes)

fig, ax = plt.subplots(figsize=(12, 10))
cax = ax.matshow(conf_mat, cmap='Blues')

plt.colorbar(cax)

for i in range(conf_mat.shape[0]):
    for j in range(conf_mat.shape[1]):
        ax.text(j, i, str(conf_mat[i, j]),
                va='center', ha='center', color='black')

plt.xlabel('Predicted Classes')
plt.ylabel('Actual Classes')

ax.set_xticks(np.arange(len(class_names)))
ax.set_yticks(np.arange(len(class_names)))

ax.set_xticklabels(class_names)
ax.set_yticklabels(class_names)

plt.xticks(rotation=90)
plt.yticks(rotation=0)

plt.tight_layout()

plt.show()

## Plot Training/Validation Accuracy and Loss


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history['accuracy'], label='Train Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()